In [0]:
# ============================================
# BRONZE LAYER - CARGA DE DADOS BRUTOS
# ============================================
# Objetivo: Copiar dados raw para Delta Lake
# sem transformações, apenas auditoria
# ============================================

from pyspark.sql import functions as F
from datetime import datetime

SOURCE_TABLE = "default.extratos_bancarios"
TARGET_TABLE = "finance_bronze.transacoes_bronze"

print("Bibliotecas importadas ok")
print(f"Origem: {SOURCE_TABLE}")
print(f"Destino: {TARGET_TABLE}")

In [0]:
# ============================================
# 1. CARREGAR DADOS RAW
# ============================================

df_raw = spark.read.table(SOURCE_TABLE)

print(f"Linhas carregadas: {df_raw.count()}")
print(f"Colunas: {df_raw.columns}")
print("\n Schema dos dados:")
df_raw.printSchema()

print("\n Amostra dos dados:")
display(df_raw.limit(5))

In [0]:
# ============================================
# 2. ADICIONAR CAMPOS DE AUDITORIA
# ============================================
# Bronze = dados brutos + auditoria
# NÃO fazemos transformações aqui

df_bronze = df_raw.withColumn(
    "load_timestamp",
    F.current_timestamp()
).withColumn(
    "source_file", 
    F.lit("extratos_bancarios.csv")
).withColumn(
    "bronze_id",
    F.monotonically_increasing_id()
)

print(" Campos de auditoria adicionados:")
print(f"   - load_timestamp: timestamp atual")
print(f"   - source_file: origem dos dados")
print(f"   - bronze_id: ID único sequencial")

print(f"\n Total de colunas agora: {len(df_bronze.columns)}")
print(f" Colunas: {df_bronze.columns}")

print("\n Amostra com auditoria:")
display(df_bronze.select(
    "bronze_id", 
    "data", 
    "descricao", 
    "valor", 
    "load_timestamp", 
    "source_file"
).limit(3))

In [0]:
# ============================================
# 3. SALVAR COMO DELTA TABLE
# ============================================

print(" Salvando dados na camada Bronze...")
print(f" Destino: {TARGET_TABLE}")

df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(TARGET_TABLE)

print(" Tabela Bronze foi criada ok")
print(f" Localização: {TARGET_TABLE}")

bronze_count = spark.read.table(TARGET_TABLE).count()
print(f" Linhas na tabela Bronze: {bronze_count}")

In [0]:
# ============================================
# 4. VALIDAÇÃO E ESTATÍSTICAS
# ============================================

print(" VALIDAÇÃO DA CAMADA BRONZE\n")

df_validation = spark.read.table(TARGET_TABLE)

total = df_validation.count()
print(f"1️. Total de registros: {total:,}")

periodo = df_validation.select(
    F.min("data").alias("data_inicial"),
    F.max("data").alias("data_final")
).collect()[0]
print(f"2️. Período: {periodo.data_inicial} até {periodo.data_final}")

print("\n3️. Distribuição por tipo:")
df_validation.groupBy("tipo") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

print("4️. Distribuição por categoria:")
df_validation.groupBy("categoria") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

print("5️. Estatísticas de valores:")
df_validation.select(
    F.min("valor").alias("valor_minimo"),
    F.max("valor").alias("valor_maximo"),
    F.avg("valor").alias("valor_medio"),
    F.sum("valor").alias("valor_total")
).show()

print("\n CAMADA BRONZE CONCLUÍDA OK")